In [ ]:
!pip --quiet install pymystem3 nltk gensim kneed faiss-cpu rouge tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 31.9 MB/s eta 0:00:00


In [ ]:
import nltk
import faiss
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
from pymystem3 import Mystem
from gensim.models import KeyedVectors
from google.colab import drive
from sklearn.cluster import DBSCAN
from tqdm import tqdm

drive.mount('/content/drive')
nltk.download('stopwords')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
# Defining stopwords
russian_stopwords = stopwords.words('russian')

# Defining the pymystem analyzer
nlp = Mystem()

# Load the RusVectōrēs model
emb_path = '/content/drive/MyDrive/StudCamp HSE x YSDA/model.bin'
emb_model = KeyedVectors.load_word2vec_format(emb_path, binary=True)

In [ ]:
class Analyzer:

  def __init__(self, analyzer: Mystem, stopwords: list) -> None:
    self.analyzer = analyzer
    self.stopwords = stopwords

  def get_nouns(self, text: str) -> list:
    summary = self.analyzer.analyze(text)

    nouns = []
    for item in summary:
      if 'analysis' in item and item['analysis']:
        analysis = item['analysis'][0]
        if 'gr' in analysis:
          pos = analysis['gr'].split('=')[0].split(',')[0]
          if pos == 'S' and analysis['lex'] not in self.stopwords:
            nouns.append(analysis['lex'])
    return nouns

In [ ]:
class Vectorizer:

  def __init__(self, emb_model: KeyedVectors) -> None:
    self.emb_model = emb_model

  def vectorize_nouns(self, nouns: list) -> np.ndarray:
    embeddedNouns = [self.emb_model[noun + '_NOUN'] for noun in nouns if noun + '_NOUN' in self.emb_model]
    return np.array(embeddedNouns)

In [ ]:
class Clusterizer:

  def __init__(self) -> None:
    pass

  def get_clusters_centroids(self, vectorized_nouns: np.ndarray) -> list:

    labels, samples = None, None
    for parameter in range(2, int(np.sqrt(vectorized_nouns.shape[0]))):
      dbscan = DBSCAN(min_samples=parameter, eps=0.1, metric='cosine');
      model = dbscan.fit(vectorized_nouns)
      labels = model.labels_
      samples = parameter

      if len(set(labels) - {-1}) < 6:
        break

    clusters = {}
    n_clusters = len(set(labels)) - (1 if -1 in labels else 0)

    for cluster_id in range(n_clusters):
        points_in_cluster = vectorized_nouns[labels == cluster_id]
        clusters[cluster_id] = points_in_cluster

    centroids = [np.mean(embeddings, axis=0) for key, embeddings in clusters.items()]
    return centroids

In [ ]:
class FaissKeywordExtractor:

  def __init__(self, emb_model: KeyedVectors) -> None:
    self.word_vectors = np.array([emb_model[word] for word in emb_model.key_to_index.keys()])
    self.words = list(emb_model.key_to_index.keys())
    self.index = faiss.IndexFlatL2(self.word_vectors.shape[1])
    self.index.add(self.word_vectors.astype(np.float32))

  def find_closest(self, vec: np.ndarray) -> str:
    query_vector = vec.astype(np.float32)
    _, indices = self.index.search(query_vector.reshape(1, -1), 1)
    most_similar_word = self.words[indices[0][0]]
    return most_similar_word

  def get_tags(self, centroids: list) -> list:
    tags = [self.find_closest(centroid).split('_')[0] for centroid in centroids]
    return tags

In [ ]:
text1 = """ Яндекс открыл набор на студкемпы — бесплатные сверхинтенсивные программы для студентов IT-специальностей. За две недели студенты изучат материал, на освоение которого в рамках традиционных программ уходит от пары месяцев до нескольких семестров. Они получат фундаментальные знания в области искусственного интеллекта, а также познакомятся с практиками применения нейросетей в сервисах Яндекса. В 2024 году пройдут четыре очных студкемпа, участвовать в которых могут студенты вузов из всех регионов России.

Каждый студкемп посвящен одной из областей компьютерных наук: разработке ПО, машинному обучению, науке о данных и искусственному интеллекту. Авторы и преподаватели — эксперты Яндекса и Школы анализа данных, а также исследователи и преподаватели ведущих российских вузов. Обучение проходит на площадках партнёров: НИУ ВШЭ, Университет ИТМО, Университет Иннополис и УрФУ.

С 1 по 13 апреля в Москве пройдет студкемп по машинному обучению на базе факультета компьютерных наук НИУ ВШЭ. Участники познакомятся с современными подходами в NLP и глубоком обучении, освоят методы сбора данных, в том числе с помощью YandexGPT, и научатся визуализировать результаты при помощи фреймворков. Во время итогового проекта студенты создадут MVP системы для решения задачи обработки естественного языка.
"""

In [ ]:
# Instansiate classes
analyzer = Analyzer(nlp, russian_stopwords)
vectorizer = Vectorizer(emb_model)
clusterizer = Clusterizer()
tagExtractor = FaissKeywordExtractor(emb_model)

In [ ]:
# Get tags
nouns = analyzer.get_nouns(text1)
vectorized_nouns = vectorizer.vectorize_nouns(nouns)
cluster_centroids = clusterizer.get_clusters_centroids(vectorized_nouns)
tags = tagExtractor.get_tags(cluster_centroids)
tags

# Тестирование

In [ ]:
import ast
import numpy as np
from rouge import Rouge
from tqdm import tqdm
from typing import Dict, List

# Downloading dataset
data = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/new_data.csv")
X_test = data["text"]
y_test = data["tag"].apply(ast.literal_eval).to_list()

In [ ]:
# Defining metric
def rogue_score_corpus(true: List[List[str]], pred: List[List[str]]) -> Dict[str, float]:
    """Calculate ROGUE-1 (precision, recall and f1-score)

    `Recall = num_words_matches / num_words_in_reference`

    `Precision = num_words_matches / num_words_in_summary`

    `F1 = classic f1 score`

    Args:
        true (List[List[str]]): True list with lists of tags for texts
        pred (List[List[str]]): Predicted tags for texts

    Returns:
        Dict[str, float]: {"recall": num, "precision": num, "f1": num}
    """

    rouge = Rouge()
    rec, prec, f1 = list(), list(), list()

    for true_tags, pred_tags in tqdm(zip(true, pred)):
        if len(true_tags) == 0 or len(pred_tags) == 0:
            continue

        true_tags_str = " ".join(true_tags)
        pred_tags_str = " ".join(pred_tags)

        scores = rouge.get_scores(pred_tags_str, true_tags_str)
        rogue_1 = scores[0]["rouge-1"]

        rec.append(rogue_1["r"])
        prec.append(rogue_1["p"])
        f1.append(rogue_1["f"])

    return {
        "recall": np.mean(rec),
        "precision": np.mean(prec),
        "f1": np.mean(f1),
    }

In [ ]:
# Instansiate classes
analyzer = Analyzer(nlp, russian_stopwords)
vectorizer = Vectorizer(emb_model)
clusterizer = Clusterizer()
tagExtractor = FaissKeywordExtractor(emb_model)

In [ ]:
tags = []
for text in tqdm(X_test):
  nouns = analyzer.get_nouns(text)
  vectorized_nouns = vectorizer.vectorize_nouns(nouns)
  cluster_centroids = clusterizer.get_clusters_centroids(vectorized_nouns)
  cur_tags = tagExtractor.get_tags(cluster_centroids)
  tags.append(cur_tags)
rogue_score_corpus(y_test, tags)

100%|██████████| 3108/3108 [19:24<00:00,  2.67it/s]


3108it [00:00, 11099.53it/s]
{'recall': 0.17428784928784924,
 'precision': 0.11730710302138872,
 'f1': 0.1377398483478611}